I get the list of NASDAQ-traded stocks from CRSP tool (permno.txt). I then input that into CRSP to get the corresponding price data (no fundamentals or sectoral info available from CRSP). That is crsp_data; I then exclude the stocks never traded on the NASDAQ to generate checked_permno.txt. I then input this into CRSP-Compustat merged.

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from IPython.display import clear_output

In [3]:
def load_data(path):
    df = pd.read_csv(path)
    df = df.rename(columns={"datadate": "Date",
                            "tic":"Ticker","conm":"Company", "datacqtr":"Calendar Quarter","datafqtr":"Fiscal Quarter", 
                            "dpq":"Depreciation and Amortisation",
                            "wcapq":"Working Capital",
                            "ggroup":"Group","gind":"Industry","gsector":"Sector","gsubind":"Subindustry",
                            "cshopq":"Total Shares Repurchased", "cshoq":"Common Shares Outstanding",
                            "dlcq":"Debt in Current Liabilities","dlttq":"Total Long Term Debt",
                            "niq":"Net Income","dltisy":"Long Term Debt Issuance YTD",
                            "dltry":"Long Term Debt Reduction YTD",
                            "aqcy":"Acquisitions YTD","capxy": "Capital Expenditures YTD",
                            "dvpq":"Preferred Dividends", "saleq":"Sales", "ceqq":"Book Value",#i.e. common equity
                            "xrdq":"R&D Expenses",
                            "cheq":"Cash and Short Term Investments",
                            "cshtrq":"Common Shares Traded",
                            "dvpsxq":"Dividends","prccq":"Price"}) # dividends per share ex-date
                            # for rename, extra labels listed don’t throw an error by default.
    
    df = df[df["Price"].notna()]
    df["Date"]=pd.to_datetime(df["Date"], format='%Y%m%d')
    
    df= df.sort_values(by=['LPERMNO','Date','fqtr'])
    
    df = df[(df["Net Income"].notna())|(df["Dividends"]!=0)]
    df = df.groupby(["LPERMNO"]).filter(lambda x: len(x) > 8) # minimum length to run bubble tests should be 9 
    df = df.drop_duplicates(subset = ["LPERMNO","Calendar Quarter"], keep ="last")
    df = df.drop_duplicates(subset = ["LPERMNO","Date"], keep ="last").reset_index(drop = True)
    # this is robust- I sorted by fiscal quarter (i.e. 1,2,3,4) above and it is indeed best to keep the last value.
    # These duplicates seem to be the same in every way except for the fiscal quarter
    # and the row with the lastest fiscal quarter seems to be the most complete.
    
    df = df.drop(columns = ["indfmt","consol","popsrc","datafmt","exchg","curcdq","costat","fyearq","mkvaltq","invchy","actq","lctq"],errors='ignore')
     #drop market value because it's mostly missing. Just use shares outstanding instead.
    # invchy after adjusted for ytd is just the decrease in total inventories invtq, same thing.
                        
    return df


In [4]:
df = load_data(r"compustat data v4.csv")

Sorting out sectors.

In [5]:
for permno, group in df.groupby("LPERMNO"):
    df.loc[df.LPERMNO==permno,"Sector"]= group["Sector"].iloc[-1]
    # on rare occasions a company changes sector.
    # there's only 30 cases like this or something so I just make each company's sector the last one it was in.
    
for permno, group in df.groupby("LPERMNO"):
    if len(group["Sector"].unique())>1:
        print(permno)
        
for i in ["Sector","Group","Industry","Subindustry"]:
    df[i].fillna(56,inplace=True)
    df[i] = df[i].astype(int).astype(str)
    df[i] = df[i].replace("56","Other")
df["Sector"].replace({"45":"Information Technology","50":"Communication Services","20":"Industrials",
              "25":"Consumer Discretionary","35":"Healthcare","30":"Consumer Staples","15":"Materials",
              "55":"Utilities","10":"Energy","60":"Real Estate","40":"Financials","Other":"Other"},inplace = True)


Cross-check stock exchange data between CRSP and CCM. I also checked: using compustat bank fundamentals gives the same underlying statistics. Now, banks do have other factors but that is beyond the scope of this study.

In [7]:
df2 = pd.read_csv(r"crsp data.csv")
df2 = df2.rename(columns={"date": "Date", "EXCHCD": "Exchange Code",
                          "TICKER":"Ticker","COMNAM":"Company",
                          "DIVAMT":"Dividend","PRC":"Price","VOL":"Volume","SHROUT":"Shares Outstanding"
                         })
df2 = df2[df2["Price"].notna()]
df2 = df2[df2["Exchange Code"]==3]
df2["Dividend"] = df2["Dividend"].fillna(0)
# CRSP's hyphenated price values means there was no close price, but instead a bid/ask average.
df2["Price"]= df2["Price"].abs()

df2["Date"]=pd.to_datetime(df2["Date"], format='%Y%m%d', errors='ignore')

df2= df2.sort_values(by=['PERMNO','Date','Dividend']) 
# things are mostly ordered, but I want to sort by dividend too in order to drop duplicates and keep the largest dividend entry.

df2 = df2.drop_duplicates(subset = ["PERMNO","Date"], keep ="last").reset_index(drop = True)

j = 0
length = len(df["LPERMNO"].unique())

df["Exchange Check"] = 0

for permno, group in df.groupby("LPERMNO"):
    clear_output(wait=True)
    match = df2[df2["PERMNO"]==permno]
    firstdate = match["Date"].iloc[0]
    lastdate = match["Date"].iloc[-1]
    # aim here is to remove all observations that weren't when the stock was listed on the NASDAQ
    df.loc[(df.LPERMNO ==permno) & (df.Date < firstdate),"Exchange Check"] = 1
    df.loc[(df.LPERMNO ==permno) & (df.Date > lastdate),"Exchange Check"]=1 # these are the observations I want to drop!
    # this is much quicker than dropping observations in place
    j+=1
    print(j,"/",length)
    
df = df[df["Exchange Check"]==0].reset_index(drop = True)
    
df.drop("Exchange Check",axis=1,inplace=True)

11544 / 11544


Date cleaning.

In [ ]:
# some specific errors I found
df.loc[(df.LPERMNO ==77862)&(df.Date =="1993-04-30"),"Calendar Quarter"] = "1993Q1"
df.loc[df.LPERMNO ==11453,"Calendar Quarter"] = df["Fiscal Quarter"]
df.loc[(df.LPERMNO ==11665)&(df.Date=="1993-04-30"),"Calendar Quarter"] = "1993Q1"
df.loc[(df.LPERMNO ==11665)&(df.Date=="1993-04-30"),"fqtr"] = 1
df.loc[(df.LPERMNO ==60281)&(df.Date=="1975-04-30"),"Calendar Quarter"] = "1975Q1"
df.loc[(df.LPERMNO ==77040)&(df.Date=="1996-07-31"),"Calendar Quarter"] = "1996Q3"
df.loc[(df.LPERMNO ==77040)&(df.Date=="1996-10-31"),"Calendar Quarter"] = "1996Q4"
df.drop(df[(df.LPERMNO ==79672)&(df.Date=="1994-04-30")].index,inplace=True,errors="ignore")
df.loc[(df.LPERMNO ==80063)&(df.Date=="1995-04-30"),"Calendar Quarter"] = "1995Q1"

df["Dividends"] = df["Dividends"].fillna(0)
df["Preferred Dividends"] = df["Preferred Dividends"].fillna(0)

# standardising quarterly dates
df["Adjusted Date"] = pd.to_datetime(df["Calendar Quarter"])+pd.tseries.offsets.QuarterEnd(0)
# to keep the format in line with the original data, I want to have the dates at the end of each quarter.
# by default, pandas quarter conversion gives the start of the quarter.
# if it were the end of quarter, then offsetting that by 1 would actually put me in the next quarter which would be wrong.
# here, both 0 and 1 offsets work.

df.loc[df["Calendar Quarter"].isna(),"Adjusted Date"]= df["Date"] + pd.tseries.offsets.QuarterEnd(0)
df = df.drop_duplicates(subset = ["LPERMNO","Adjusted Date"], keep ="last")

# Sometimes a firm may also change accounting standards/its fiscal calendar 
# it could have a report in both January and April which makes no sense.
df["Date"]= df["Adjusted Date"]
df.drop(columns=["Adjusted Date"],inplace = True)

Script that finds breaks in series and separates them if segments are long enough, or discards if too short.

In [10]:
#removing gaps
df["Adjusted_LPERMNO"]=df["LPERMNO"].astype(str)

counter = 0
for permno, group in df.groupby("LPERMNO"):
    
    clear_output(wait=True)
    
    indices = list(group.index)
    dates =list(group["Date"])
    expected_dates = list(pd.date_range(group["Date"].min(),group["Date"].max(),freq="Q"))
    # don't need to worry about offsetting to the end of the quarter here
    # because the start and end points are already offset.
    
    if len(dates)!= len(expected_dates):
        
        start = 0
        marker = 0
        for i in range(len(dates)-1):
            
            
            if (dates[i+1]-dates[i]).days>92: 
                # max 92 days in a quarter
                # if this condition is met
                # then between indices i+1 and i there is a break.'
                
                if i- start>7:
                    
                    # so the current segment goes from index 'start' to index i.
                    # minimum segment length is 9, so the difference >=8 i.e. >7.
                    df.loc[indices[start:i+1],"Adjusted_LPERMNO"]+= str(marker)
                    marker+=1
                    # every time we get a new usable series segment, marker increments.
                    
                    # remember, everything is already sorted
                    # e.g. 10001 is split into 100010, 100011
                
                else:
                    # segment not long enough, so drop
                    df.drop(indices[start:i+1], inplace=True)
                    # both loc and drop take index arguments as opposed to iloc which takes positional arguments.
                    
                start = i+1
                # increment 'start' (as an index of 'dates') for the next segment
                
    counter+=1
    print(counter)
    
# If there is a break between the penultimate and final values, the script doesn't do anything to the final value.
#Hence I apply this:
df = df.groupby(["Adjusted_LPERMNO"]).filter(lambda x: len(x) > 8)
df = df.reset_index(drop = True)
# df["LPERMNO"]= df["Adjusted_LPERMNO"]
# df.drop(columns=["Adjusted_LPERMNO"],inplace = True)

11519


Unrelated stuff used to generate figures

In [78]:
# df4 = pd.read_excel(r"us internet.xlsx")
# df4["Date"]=pd.to_datetime(df4["Date"], format='%Y')
# fig = px.line(df4,x="Date",y="Usage")
# fig.update_layout(
#     title="US Internet Usage",
#     xaxis_title="Date",
#     yaxis_title="% of Population"
# )

# df4 = pd.read_csv(r"nasdaq composite.csv")
# df4["Date"]=pd.to_datetime(df4["Date"], format='%Y%m%d')
# fig = px.line(df4,x="Date",y="Price")
# fig.update_layout(
#     title="Nasdaq Composite",
#     xaxis_title="Date",
#     yaxis_title="Points")


Now that I've removed gaps, we can transform ytd variables to quarterly.
Lots of gaps in FCFE variables...we can run the gap removing algorithm again later.

In [12]:
ytd = ["Long Term Debt Issuance YTD","Long Term Debt Reduction YTD", "Acquisitions YTD","Capital Expenditures YTD"]
for i in ytd:
    df[i[:-4]]= df.groupby("LPERMNO")[i].diff() 
    # i.e.
    # df["Capital Expenditures"] = df.groupby("LPERMNO")["Capital Expenditures YTD"].diff()
    # i[:-4] denotes the column name

    df.loc[df.fqtr==1, i[:-4] ] = df[i]
    # it resets with each fiscal year
    # i.e. capex is same as capex ytd in first quarter, but then for q2 it's the sum of q1 and q2 capex.
    df.drop(i,axis=1,inplace=True)
    
df.loc[df["Capital Expenditures"]<0,"Capital Expenditures"] = 0
# following https://faculty.wharton.upenn.edu/wp-content/uploads/2017/09/Sikes_20170926.pdf
# who also gives ytd explanations

Calculating the components for FCFE.

In [13]:
df["Change in Current Debt"] = df.groupby("LPERMNO")["Debt in Current Liabilities"].diff()

# 1 Definition
df["Net Borrowing"] = df["Long Term Debt Issuance"]-df["Long Term Debt Reduction"] +df["Change in Current Debt"]
# i.e. increase in long-term debt + increase in current debt
# https://dro.dur.ac.uk/18174/1/18174.pdf


# 2 Definition

df["Change in Long Term Debt v2"] = df.groupby("LPERMNO")["Total Long Term Debt"].diff()
df["Net Borrowing v2"] = df["Change in Long Term Debt v2"]+df["Change in Current Debt"]

# https://www.cambridge.org/core/journals/journal-of-financial-and-quantitative-analysis/article/where-did-all-the-dollars-go-the-effect-of-cash-flows-on-capital-and-asset-structure/EE06084C564500A2996EF210D26BE54B
# these guys use this definition

# they're the same in 22% of observations....but most of the time they're not!! I don't know why.


Non-Cash Working Capital calculations:

In [14]:

df["NCWC"] = df["Working Capital"]-df["Cash and Short Term Investments"]+df["Debt in Current Liabilities"]

# define non-cash (net) working capital as 
# the difference between non-cash current assets and non-debt current liabilities (Damodaran 1999)
# so (current assets - cash) - (current liabilities - debt)

df["Change in NCWC"] = df.groupby("LPERMNO")["NCWC"].diff()

# Can also use accounts receivable, inventories, other assets, payables, taxes payable etc.:
# https://www.sciencedirect.com/science/article/pii/S0304405X99000422
# but I'm not using this definition anymore

df["Acquisitions"] = df["Acquisitions"].fillna(0)
# First, any capital expenditures, defined broadly to include acquisitions, are subtracted from the net income... (Damodaran c14)

In [15]:
#1 using net borrowing version 1
df["FCFE"] = df["Net Income"] - df["Capital Expenditures"] -df["Acquisitions"]+ df["Depreciation and Amortisation"]- df["Change in NCWC"]+df["Net Borrowing"]-df["Preferred Dividends"]
#2 using net borrowing version 2
df["FCFE v2"] = df["Net Income"] - df["Capital Expenditures"] -df["Acquisitions"]+ df["Depreciation and Amortisation"]- df["Change in NCWC"]+df["Net Borrowing v2"]-df["Preferred Dividends"]


Generating dissertation graphs:

In [19]:
# totals = []
# nodivs = []
# for sector in df["Sector"].unique():
#     total = 0
#     nodiv = 0
#     df2 = df[df.Sector ==sector]
#     for permno, group in df2.groupby("LPERMNO"):
#         total+=1
#         if (group["Dividends"]==0).all():
#             nodiv+=1
#     totals.append(total)
#     nodivs.append(nodiv)
# df2 = pd.DataFrame({"Sector":df["Sector"].unique(),"Total number of Stocks":totals,"Non-dividend paying stocks":nodivs})
# df3 = df.groupby("Sector")[["Net Income"]].describe()
# merged = pd.merge(df2,df3, left_on="Sector",right_index=True)
# merged
# merged.to_excel("no_dividends_netincome_chart.xlsx")

In [215]:
# df2 = df.groupby("Date",as_index=False).count()
# fig = px.line(df2,x="Date",y="Price")
# fig.update_layout(
#     title="Number of stocks listed on NASDAQ",
#     xaxis_title="Date",
#     yaxis_title="Number of stocks")
# fig.show()

In [57]:
# To the degree that cash cannot be invested to earn market returns, and is needed for day-to-day operations,
# it is appropriate to look at changes in net working capital, with cash included.
# https://pages.stern.nyu.edu/~adamodar/New_Home_Page/CFTheory/deriv/ch14der.html
# https://www.wallstreetoasis.com/forums/change-in-net-working-capital-formula
# https://quant.stackexchange.com/questions/37477/why-subtract-increase-in-net-working-capital-to-get-free-cash-flows

# https://sites.bu.edu/qm222projectcourse/files/2014/08/compustat_users_guide-2003.pdf
# market to book if I need it


In [25]:
df.to_csv("compustat cleaned v2.csv",index=False)

In [61]:
df.drop(columns = ['fqtr',
       'Current Assets', 'Cash and Short Term Investments',
       'Debt in Current Liabilities',
       'Total Long Term Debt', 'Depreciation and Amortisation',
       'Preferred Dividends','Current Liabilities','Working Capital', 'Long Term Debt Issuance',
       'Long Term Debt Reduction', 'Change in Current Debt',
       'Capital Expenditures', 'Net Borrowing',
       'Change in Long Term Debt v2', 'Net Borrowing v2', 'NCWC',
       'Change in NCWC'],inplace = True)

In [63]:
df.to_csv("compustat cleaned.csv",index=False)